# Import modules and create Spark Session

In [2]:
#import modules
from pyspark.ml.recommendation import ALS

In [3]:
#Create Session

appName = "Recommender System in Spark"
spark = SparkSession.builder.appName('appName').getOrCreate()

# Read file into DataFrame

In [5]:
#read file into dataFrame using automatically inferred schema
ratings = spark.read.csv('dataset/ratings.csv', inferSchema=True, header=True)
movies = spark.read.csv('dataset/movies.csv', inferSchema=True, header=True)
#merge "movies" and "ratings" dataFrame based on "movieId"
ratings.join(movies, "movieId").show(3)

+-------+------+------+----------+--------------------+--------------------+
|movieId|userId|rating| timestamp|               title|              genres|
+-------+------+------+----------+--------------------+--------------------+
|     31|     1|   2.5|1260759144|Dangerous Minds (...|               Drama|
|   1029|     1|   3.0|1260759179|        Dumbo (1941)|Animation|Childre...|
|   1061|     1|   3.0|1260759182|     Sleepers (1996)|            Thriller|
+-------+------+------+----------+--------------------+--------------------+
only showing top 3 rows



In [ ]:
# Join movies and ratings dataframe based on common column
ratings.join(movies,"movieId").show(5)

# Data Preparation

In [9]:
#use only column data of "userId", "movieId", dan "rating"
data = ratings.select("userId", "movieId", "rating")
#divide data, 70% for training and 30% for testing
splits = data.randomSplit([0.7, 0.3])
train = splits[0].withColumnRenamed("rating", "label")
test = splits[1].withColumnRenamed("rating", "trueLabel")

In [10]:
#calculate number of rows
train_rows = train.count()
test_rows = test.count()
print ("number of training data rows:", train_rows, 
       ", number of testing data rows:", test_rows)

number of training data rows: 70151 , number of testing data rows: 29853


# Define model and train it

In [12]:
#define ALS (Alternating Least Square) as our recommender system
als = ALS(maxIter=19, regParam=0.01, userCol="userId", 
          itemCol="movieId", ratingCol="label")
#train our ALS model
model = als.fit(train)
print("Training is done!")

Training is done!


In [13]:
#Predict testing data
prediction = model.transform(test)
print("testing is done!")

testing is done!


In [14]:
prediction.join(movies, "movieId").select("userId", "title", "prediction", "trueLabel").show(n=10, truncate=False)

+------+---------------------------+----------+---------+
|userId|title                      |prediction|trueLabel|
+------+---------------------------+----------+---------+
|232   |Guilty as Sin (1993)       |3.0934198 |4.0      |
|242   |Guilty as Sin (1993)       |3.46531   |4.0      |
|311   |Guilty as Sin (1993)       |3.1228745 |3.0      |
|548   |Hudsucker Proxy, The (1994)|3.423729  |4.0      |
|285   |Hudsucker Proxy, The (1994)|3.7966106 |5.0      |
|440   |Hudsucker Proxy, The (1994)|2.7109172 |3.0      |
|306   |Hudsucker Proxy, The (1994)|2.8805468 |3.0      |
|537   |Hudsucker Proxy, The (1994)|3.692778  |5.0      |
|380   |Hudsucker Proxy, The (1994)|4.1300335 |4.0      |
|624   |Hudsucker Proxy, The (1994)|3.8224726 |4.0      |
+------+---------------------------+----------+---------+
only showing top 10 rows



# Evaluate the accuracy of our model

In [15]:
#import RegressionEvaluator since we also want to calculate RMSE (Root Mean Square Error)
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): nan


In [16]:
prediction.count()
a = prediction.count()
print("number of original data rows: ", a)
#drop rows with any missing data
cleanPred = prediction.dropna(how="any", subset=["prediction"])
b = cleanPred.count()
print("number of rows after dropping data with missing value: ", b)
print("number of missing data: ", a-b)

number of original data rows:  29853
number of rows after dropping data with missing value:  28650
number of missing data:  1203


In [17]:
rmse = evaluator.evaluate(cleanPred)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 1.238084156532729
